In [ ]:
import torch
import scipy.misc
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torch import optim


import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# 構建模型（簡單的卷積神經網路）
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size =5, padding = 2) # 卷積
        self.conv2 = nn.Conv2d(6, 16, 5)
        # Linear(in_feactures(輸入的二維張量大小), out_feactures)
        self.fc1   = nn.Linear(16*5*5, 120) # 全連接
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10) # 最後輸出10個類

    def forward(self, x):
        # 激活函數
        out = F.relu(self.conv1(x))
        # max_pool2d(input, kernel_size（卷積核）, stride(卷積核步長)=None, padding=0, dilation=1, ceil_mode（空間輸入形狀）=False, return_indices=False)
        out = F.max_pool2d(out, kernel_size = 2) # 池化

        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)

        # 將多維的數據平鋪為一維
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [ ]:
def train():
    # 學習率0.001
    learning_rate = 1e-3
    # 單次大小
    batch_size = 100
    # 總的循環
    epoches = 50
    lenet = LeNet()

    # 1、數據集準備
    # 這個函數包括了兩個操作：transforms.ToTensor()將圖片轉換為張量，transforms.Normalize()將圖片進行歸一化處理
    trans_img = transforms.Compose([transforms.ToTensor()])
    # path = './data/'數據集下載後保存的目錄，下載訓練集
    trainset = MNIST('./data', train=True, transform=trans_img, download=True)
    # 構建數據集的DataLoader,
    # Pytorch自提供了DataLoader的方法來進行訓練，該方法自動將數據集打包成為迭代器，能夠讓我們很方便地進行後續的訓練處理
    # 迭代器(iterable)是一個超級接口! 是可以遍歷集合的對象,
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=10)

    # 2、構建迭代器與損失函數
    criterian = nn.CrossEntropyLoss(reduction='sum')  # loss（損失函數）
    optimizer = optim.SGD(lenet.parameters(), lr=learning_rate)  # optimizer（迭代器）

    # 如果網絡能在GPU中訓練，就使用GPU；否則使用CPU進行訓練
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #lenet.to("cpu")

    # 3、訓練
    for i in range(epoches):
        running_loss = 0.
        running_acc = 0.
        for (img, label) in trainloader:  # 將圖像和標籤傳輸進device中
            optimizer.zero_grad()  # 求梯度之前對梯度清零以防梯度累加
            output = lenet(img)  # 對模型進行前向推理
            loss = criterian(output, label)  # 計算本輪推理的Loss值
            loss.backward()    # loss反傳存到相應的變量結構當中
            optimizer.step()   # 使用計算好的梯度對參數進行更新
            running_loss += loss.item()
            #print(output)
            _, predict = torch.max(output, 1)  # 計算本輪推理的準確率
            correct_num = (predict == label).sum()
            running_acc += correct_num.item()

        running_loss /= len(trainset)
        running_acc /= len(trainset)
        print("[%d/%d] Loss: %.5f, Acc: %.2f" % (i + 1, epoches, running_loss, 100 * running_acc))

    return lenet


In [ ]:
def train():
    # 學習率0.001
    learning_rate = 1e-3
    # 單次大小
    batch_size = 100
    # 總的循環
    epoches = 50
    lenet = LeNet()

    # 1、數據集準備
    # 這個函數包括了兩個操作：transforms.ToTensor()將圖片轉換為張量，transforms.Normalize()將圖片進行歸一化處理
    trans_img = transforms.Compose([transforms.ToTensor()])
    # path = './data/'數據集下載後保存的目錄，下載訓練集
    trainset = MNIST('./data', train=True, transform=trans_img, download=True)
    # 構建數據集的DataLoader,
    # Pytorch自提供了DataLoader的方法來進行訓練，該方法自動將數據集打包成為迭代器，能夠讓我們很方便地進行後續的訓練處理
    # 迭代器(iterable)是一個超級接口! 是可以遍歷集合的對象,
    trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=10)

    # 2、構建迭代器與損失函數
    criterian = nn.CrossEntropyLoss(reduction='sum')  # loss（損失函數）
    optimizer = optim.SGD(lenet.parameters(), lr=learning_rate)  # optimizer（迭代器）

    # 如果網絡能在GPU中訓練，就使用GPU；否則使用CPU進行訓練
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #lenet.to("cpu")

    # 3、訓練
    for i in range(epoches):
        running_loss = 0.
        running_acc = 0.
        for (img, label) in trainloader:  # 將圖像和標籤傳輸進device中
            optimizer.zero_grad()  # 求梯度之前對梯度清零以防梯度累加
            output = lenet(img)  # 對模型進行前向推理
            loss = criterian(output, label)  # 計算本輪推理的Loss值
            loss.backward()    # loss反傳存到相應的變量結構當中
            optimizer.step()   # 使用計算好的梯度對參數進行更新
            running_loss += loss.item()
            #print(output)
            _, predict = torch.max(output, 1)  # 計算本輪推理的準確率
            correct_num = (predict == label).sum()
            running_acc += correct_num.item()

        running_loss /= len(trainset)
        running_acc /= len(trainset)
        print("[%d/%d] Loss: %.5f, Acc: %.2f" % (i + 1, epoches, running_loss, 100 * running_acc))

    return lenet


In [ ]:
def test(lenet):
    batch_size = 100
    trans_img = transforms.Compose([transforms.ToTensor()])
    testset = MNIST('./data', train=False, transform=trans_img, download=True)
    testloader = DataLoader(testset, batch_size, shuffle=False, num_workers=10)
    running_acc = 0.
    for (img, label) in testloader:
        output = lenet(img)
        _, predict = torch.max(output, 1)
        correct_num = (predict == label).sum()
        running_acc += correct_num.item()
    running_acc /= len(testset)
    return running_acc

In [ ]:
if __name__ == '__main__':
    lenet = train()
    torch.save(lenet, 'lenet.pkl') # save model

    lenet = torch.load('lenet.pkl') # load model
    test_acc = test(lenet)
    print("Test Accuracy:Loss: %.2f" % test_acc)

[1/50] Loss: 0.59952, Acc: 80.94
[2/50] Loss: 0.09087, Acc: 97.17
[3/50] Loss: 0.06268, Acc: 98.00
[4/50] Loss: 0.04976, Acc: 98.40
[5/50] Loss: 0.04092, Acc: 98.72
[6/50] Loss: 0.03480, Acc: 98.91
[7/50] Loss: 0.03000, Acc: 99.04
[8/50] Loss: 0.02598, Acc: 99.16
[9/50] Loss: 0.02263, Acc: 99.25
[10/50] Loss: 0.01804, Acc: 99.44
[11/50] Loss: 0.01786, Acc: 99.42
[12/50] Loss: 0.01487, Acc: 99.50
[13/50] Loss: 0.01293, Acc: 99.58
[14/50] Loss: 0.01153, Acc: 99.61
[15/50] Loss: 0.00952, Acc: 99.70
[16/50] Loss: 0.00895, Acc: 99.73
[17/50] Loss: 0.00900, Acc: 99.70
[18/50] Loss: 0.00638, Acc: 99.82
[19/50] Loss: 0.00613, Acc: 99.81
[20/50] Loss: 0.00579, Acc: 99.83
[21/50] Loss: 0.00658, Acc: 99.80
[22/50] Loss: 0.00457, Acc: 99.85
[23/50] Loss: 0.00339, Acc: 99.90
[24/50] Loss: 0.00278, Acc: 99.93
[25/50] Loss: 0.00272, Acc: 99.91
[26/50] Loss: 0.00343, Acc: 99.90
[27/50] Loss: 0.00184, Acc: 99.95
[28/50] Loss: 0.00102, Acc: 99.98
[29/50] Loss: 0.00106, Acc: 99.98
[30/50] Loss: 0.00062, 

<ipython-input-11-cbdbd06e3957>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lenet = torch.load('lenet.pkl') # load model


Test Accuracy:Loss: 0.99


In [ ]:
def practice(img_path):
    img = Image.open(img_path)
    img = img.convert('L')
    prac_img = transforms.Compose([transforms.Resize((28, 28)),transforms.ToTensor()])
    pracset = MNIST('./data', train=True, transform=prac_img, download=True)
    img = prac_img(img)
    img = torch.reshape(img, (1, 1, 28, 28))
    lenet = torch.load('lenet.pkl')  # load model
    output = lenet(img)
    output = output.argmax(1)
    dict_target = pracset.class_to_idx
    dict_target = [indx for indx, vale in dict_target.items()]  # 获得标签字典
    print('识别类型为{}'.format(dict_target[output]))

In [ ]:
from google.colab import files
uploaded = files.upload()  # 會彈出上傳文件的窗口


Saving 0.jpeg to 0.jpeg
Saving 1.jpeg to 1.jpeg
Saving 2.jpeg to 2.jpeg
Saving 3.jpeg to 3.jpeg
Saving 4.jpeg to 4.jpeg
Saving 5.jpeg to 5.jpeg
Saving 6.jpeg to 6.jpeg
Saving 7.jpeg to 7.jpeg
Saving 8.jpeg to 8.jpeg
Saving 9.jpeg to 9.jpeg


In [ ]:
if __name__ == '__main__':
    practice('0.jpeg')
    practice('1.jpeg')
    practice('2.jpeg')
    practice('3.jpeg')
    practice('4.jpeg')
    practice('5.jpeg')
    practice('6.jpeg')
    practice('7.jpeg')
    practice('8.jpeg')
    practice('9.jpeg')

<ipython-input-24-f03a62da09b8>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  lenet = torch.load('lenet.pkl')  # load model


识别类型为0 - zero
识别类型为1 - one
识别类型为2 - two
识别类型为3 - three
识别类型为4 - four
识别类型为5 - five
识别类型为6 - six
识别类型为7 - seven
识别类型为8 - eight
识别类型为9 - nine
